In [33]:
# some exploratory data analysis
from datetime import datetime
from dateutil.relativedelta import relativedelta
import json
import os

import pandas as pd
# set pandas to show 100 the columns
pd.set_option('display.max_columns', 100)

import requests


In [34]:
"""
1. get the data from S3
2. transform the weather data (weather_data)
3. transform the taxi data (taxi_trips) - DONE
4. update payment type master table (payment_type_master) - DONE
5. update company master table (company_master) - DONE
6. update the ids in the taxi_trips using the latest master tables - DONE
7. upload weather data fo S3 (weather_data)
8. upload taxi data fo S3 (taxi_trips)
9. upload the new payment and company master tables (payment_type_master, company_master)
"""

'\n1. get the data from S3\n2. transform the weather data (weather_data)\n3. transform the taxi data (taxi_trips) - DONE\n4. update payment type master table (payment_type_master) - DONE\n5. update company master table (company_master) - DONE\n6. update the ids in the taxi_trips using the latest master tables - DONE\n7. upload weather data fo S3 (weather_data)\n8. upload taxi data fo S3 (taxi_trips)\n9. upload the new payment and company master tables (payment_type_master, company_master)\n'

#### 3. transform the taxi data (taxi_trips)

In [35]:
# get the taxi data of the last full month (T-1 months') data
current_datetime = datetime.now()    # current date
# get the data two months before as a formatted string
formatted_datetime = (current_datetime - relativedelta(months=2)).strftime('%Y-%m-%d')

url = f'https://data.cityofchicago.org/resource/wrvz-psew.json?$where=trip_start_timestamp >= "{formatted_datetime}T00:00:00" AND trip_start_timestamp <= "{formatted_datetime}T23:59:59"&$limit=30000'

# the os.environ.get looks for the specified variable in the .env file of the root
# folder of the project
headers = {'X-App-Token': os.environ.get("CHICHAGO_API_TOKEN")}

# response = requests.get(url, headers)    # in case an error comes leave the headers parameter
response = requests.get(url)

data = response.json()


In [36]:
# create a dataframe from the taxi trip data
taxi_trips = pd.DataFrame(data)

In [37]:
taxi_trips.columns

Index(['trip_id', 'taxi_id', 'trip_start_timestamp', 'trip_end_timestamp',
       'trip_seconds', 'trip_miles', 'pickup_community_area',
       'dropoff_community_area', 'fare', 'tips', 'tolls', 'extras',
       'trip_total', 'payment_type', 'company', 'pickup_centroid_latitude',
       'pickup_centroid_longitude', 'pickup_centroid_location',
       'dropoff_centroid_latitude', 'dropoff_centroid_longitude',
       'dropoff_centroid_location', 'pickup_census_tract',
       'dropoff_census_tract'],
      dtype='object')

In [38]:
# # the order is important, because these two columns contain lots of NaN values
# # first drop them, then the rows having NaN in the other columns only
# taxi_trips.drop(['pickup_census_tract', 'dropoff_census_tract'], axis=1, inplace=True)
# taxi_trips.drop(['pickup_centroid_location', 'dropoff_centroid_location'], axis=1, inplace=True)
# taxi_trips.dropna(inplace=True)
# taxi_trips.rename(columns={"pickup_community_area": "pickup_community_area_id",
#                            "dropoff_community_area": "dropoff_community_area_id"},
#                            inplace=True)
taxi_trips['trip_start_timestamp'] = pd.to_datetime(taxi_trips['trip_start_timestamp']).dt.floor('H')
taxi_trips['datetime_for_weather'] = taxi_trips['trip_start_timestamp'].dt.floor('H')

#### taxi_trips transformations function

In [39]:
def taxi_trips_transformations(taxi_trips: pd.DataFrame) -> pd.DataFrame:
    """ Perform transformation with the taxi data

    Args:
        taxi_trips (pd.DataFrame): 
            The DataFrame holding the daily taxi trips.

    Returns:
        pd.DataFrame:
            The cleaned, DataFrame holding the daily taxi trips.
    """
    
    # error handling - can be extended with some other ideas
    if not isinstance(taxi_trips, pd.DataFrame):
        raise TypeError('taxi_trips is a not a valid Pandas DataFrame.')
    
    # the order is important, because these two columns contain lots of NaN values
    # first drop them, then the rows having NaN in the other columns only
    taxi_trips.drop(['pickup_census_tract','dropoff_census_tract',
                     'pickup_centroid_location', 'dropoff_centroid_location'],
                    axis=1, inplace=True)
    taxi_trips.dropna(inplace=True)
    taxi_trips.rename(columns={"pickup_community_area": "pickup_community_area_id",
                            "dropoff_community_area": "dropoff_community_area_id"},
                            inplace=True)
    taxi_trips['trip_start_timestamp'] = pd.to_datetime(taxi_trips['trip_start_timestamp']).dt.floor('H')
    
    return taxi_trips

In [40]:
taxi_trips_transformations(taxi_trips)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,payment_type,company,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather
0,93e01e2b0c688480d983d73fb2d9eaa45ecd8c76,02ef8f01232b1b1828f4e5e1b8e8a85cd71b67c449afaf...,2023-11-30 23:00:00,2023-12-01T00:00:00.000,833,10.74,69,8,27.75,0,0,0,27.75,Prcard,City Service,41.763246799,-87.616134111,41.899602111,-87.633308037,2023-11-30 23:00:00
1,fd158814e9e34cb1e75c6085942b9a9daddd46e7,d602932962e0d1a8ab0c959b43cd9da0f870c5fcc4f643...,2023-11-30 23:00:00,2023-12-01T00:00:00.000,1297,5.8,56,59,20,0,0,4.5,24.5,Cash,Chicago Taxicab,41.79259236,-87.769615453,41.829922304,-87.672502646,2023-11-30 23:00:00
2,fb2567f039bc763fced16272fba3fa0bf3123f89,9e402f28d4da35014d6f3171cf85336476cebfccf79048...,2023-11-30 23:00:00,2023-12-01T00:00:00.000,1440,17.3,76,8,43.5,12,0,4,59.5,Credit Card,Chicago Independents,41.980264315,-87.913624596,41.899602111,-87.633308037,2023-11-30 23:00:00
3,f526030a28abc9c9e7937665b20afac3c0ac085d,e5e1bb9c3329c0f9bd1f291cb9bbbb016731c148fefca8...,2023-11-30 23:00:00,2023-12-01T00:00:00.000,1009,11.57,41,6,29.5,0,0,0,29.5,Prcard,Flash Cab,41.794090253,-87.592310855,41.944226601,-87.655998182,2023-11-30 23:00:00
4,ec1fcda6af7099054030b2a368ca0743d9c60516,48c0b5669ed50a0dcd8bf0e69fd99bb2669cb027ec06c0...,2023-11-30 23:00:00,2023-12-01T00:15:00.000,1467,10.87,33,66,29.5,0,0,0,29.5,Prcard,Flash Cab,41.857183858,-87.620334624,41.771848515,-87.695666342,2023-11-30 23:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20526,6a4ce13edd7b6e856e7e85622bb9c33537c56691,008dda45db57cb6daa679a86ce8c8149ddc05446d545b1...,2023-11-30 00:00:00,2023-11-30T00:00:00.000,60,0,33,33,3.25,9.95,0,46,59.2,Credit Card,Taxi Affiliation Services,41.857183858,-87.620334624,41.857183858,-87.620334624,2023-11-30 00:00:00
20527,60723fbf301d09cb4d65c07b80d2c27159ba9e1f,90d83b1a646ae05a1b32e9a0ee7a13e01a41d34ae8c803...,2023-11-30 00:00:00,2023-11-30T00:00:00.000,120,0.1,8,7,3.75,0,0,0,3.75,Cash,Chicago City Taxi Association,41.899602111,-87.633308037,41.922686284,-87.649488729,2023-11-30 00:00:00
20528,3a7e38f038bac91b592202c352684c0bd170d43c,1f075315f5906824cc9566de5f650935e00f9e027a96a7...,2023-11-30 00:00:00,2023-11-30T00:30:00.000,1320,14.68,76,24,36.25,10.44,0,5,52.19,Credit Card,5 Star Taxi,41.980264315,-87.913624596,41.901206994,-87.676355989,2023-11-30 00:00:00
20529,6204026c7ddc2125afd8c9efccac69a1f1ad23da,e64c125ab06441db8101d7c98eec723e18c3dcd1cf39f8...,2023-11-30 00:00:00,2023-11-30T00:15:00.000,240,0.5,8,32,5,2,0,0,7,Credit Card,Taxi Affiliation Services,41.892507781,-87.626214906,41.884987192,-87.620992913,2023-11-30 00:00:00


#### company update codes

In [41]:
company_master = taxi_trips['company'].drop_duplicates().reset_index(drop=True)


company_master = pd.DataFrame(
    {
        'company_id': range(1, len(company_master)+1),    # the id's usually starts from 1 despite indeces starting from 0
        'company': company_master
    }
)

company_master

,company_id,company
0,1,City Service
1,2,Chicago Taxicab
2,3,Chicago Independents
3,4,Flash Cab
4,5,Sun Taxi
5,6,Choice Taxi Association
6,7,Taxi Affiliation Services
7,8,Medallion Leasin
8,9,5 Star Taxi
9,10,Star North Taxi Management Llc


In [42]:
# how to extend the company_master table if a new type of payment appears?
new_company_data = [
    {'company': 'Petani Cab Corp'},
    {'company': 'X'},
    {'company': 'Y'}
]
new_company_mapping = pd.DataFrame(new_company_data)

In [43]:
company_max_id = company_master['company_id'].max()

In [44]:
# get the list of new companies
list_of_new_companies = []

for company in new_company_mapping['company'].values:   # go through the list of the latest list of companies
    if company not in company_master['company'].values: # find if any is missing from our master table
        list_of_new_companies.append(company)           # add it to the list of new companies

# the same in one line
list_of_new_companies_one_line = [company for company in new_company_mapping['company'].values
                                  if company not in company_master['company'].values]
list_of_new_companies_one_line

['X', 'Y']

In [45]:
new_companies_df = pd.DataFrame({
    'company_id': range(company_max_id + 1, company_max_id + len(list_of_new_companies_one_line) + 1),
    'company': list_of_new_companies_one_line
})
new_companies_df

,company_id,company
0,30,X
1,31,Y


In [46]:
updated_company_master = pd.concat([company_master, new_companies_df], ignore_index=True)
updated_company_master.tail()

,company_id,company
26,27,6574 - Babylon Express Inc.
27,28,3556 - 36214 RC Andrews Cab
28,29,5167 - 71969 5167 Taxi Inc
29,30,X
30,31,Y


In [47]:
def update_company_master(taxi_trips: pd.DataFrame, company_master: pd.DataFrame) -> pd.DataFrame :
    """ Extend the company master with new companies if there are any of them.

    Args:
        taxi_trips (pd.DataFrame):
            DataFrame holding the daily taxi trips.
        company_master (pd.DataFrame):
            DataFrame holding the company_master data.

    Returns:
        pd.DataFrame:
            The updated company master data. If no new company appeared returns the original one.
    """
    company_max_id = company_master['company_id'].max()
    list_of_new_companies_one_line = [company for company in taxi_trips['company'].values
                                  if company not in company_master['company'].values]
    new_companies_df = pd.DataFrame({
        'company_id': range(company_max_id + 1, company_max_id + len(list_of_new_companies_one_line) + 1),
        'company': list_of_new_companies_one_line
        })
    updated_company_master = pd.concat([company_master, new_companies_df], ignore_index=True)
    
    return updated_company_master

In [48]:
taxi_trips_company_only = pd.DataFrame({
    'company_id': range(1,4),
    'company':     ['Petani Cab Corp', 'X', 'Y'] 
})
taxi_trips_company_only

,company_id,company
0,1,Petani Cab Corp
1,2,X
2,3,Y


In [49]:
updated_company_master = update_company_master(taxi_trips=taxi_trips_company_only, company_master=company_master)
updated_company_master

,company_id,company
0,1,City Service
1,2,Chicago Taxicab
2,3,Chicago Independents
3,4,Flash Cab
4,5,Sun Taxi
5,6,Choice Taxi Association
6,7,Taxi Affiliation Services
7,8,Medallion Leasin
8,9,5 Star Taxi
9,10,Star North Taxi Management Llc


#### payment type update codes

In [50]:
payment_type_master = taxi_trips['payment_type'].drop_duplicates().reset_index(drop=True)


payment_type_master = pd.DataFrame(
    {
        'payment_type_id': range(1, len(payment_type_master)+1),    # the id's usually starts from 1 despite indeces starting from 0
        'payment_type': payment_type_master
    }
)


taxi_trips_payment_type_only = pd.DataFrame({
    'payment_type_id': range(1,4),
    'payment_type':     ['Credit Card', 'X', 'Y'] 
})
payment_type_master
taxi_trips_payment_type_only

,payment_type_id,payment_type
0,1,Credit Card
1,2,X
2,3,Y


In [51]:
def update_payment_type_master(taxi_trips: pd.DataFrame, payment_type_master: pd.DataFrame) -> pd.DataFrame :
    """ Extend the payment_type master with new payment types if there are any of them.

    Args:
        taxi_trips (pd.DataFrame):
            DataFrame holding the daily taxi trips.
        payment_type_master (pd.DataFrame):
            DataFrame holding the payment_type_master data.

    Returns:
        pd.DataFrame:
            The updated payment_type master data. If no new payment type appeared returns the original one.
    """
    payment_type_max_id = payment_type_master['payment_type_id'].max()
    list_of_new_payment_types_one_line = [payment_type for payment_type in taxi_trips['payment_type'].values
                                  if payment_type not in payment_type_master['payment_type'].values]
    new_payment_types_df = pd.DataFrame({
        'payment_type_id': range(payment_type_max_id + 1, payment_type_max_id + len(list_of_new_payment_types_one_line) + 1),
        'payment_type': list_of_new_payment_types_one_line
        })
    updated_payment_type_master = pd.concat([payment_type_master, new_payment_types_df], ignore_index=True)
    
    return updated_payment_type_master

In [52]:
taxi_trips['payment_type']

0             Prcard
1               Cash
2        Credit Card
3             Prcard
4             Prcard
            ...     
20526    Credit Card
20527           Cash
20528    Credit Card
20529    Credit Card
20531    Credit Card
Name: payment_type, Length: 18717, dtype: object

In [53]:
updated_payment_type_master = update_payment_type_master(taxi_trips=taxi_trips_payment_type_only, payment_type_master=payment_type_master)
updated_payment_type_master

,payment_type_id,payment_type
0,1,Prcard
1,2,Cash
2,3,Credit Card
3,4,Mobile
4,5,Unknown
5,6,No Charge
6,7,Dispute
7,8,X
8,9,Y


#### combine the update functions to a more generic one

In [54]:
def update_master(taxi_trips: pd.DataFrame, master: pd.DataFrame, id_column: str,
                  value_column: str) -> pd.DataFrame :
    """ Extend the master with new types if there are any of them.

    Args:
        taxi_trips (pd.DataFrame):
            DataFrame holding the daily taxi trips.
        master (pd.DataFrame):
            DataFrame holding the master data.
        id_column (str):
            The id column of the master DataFrame.
        value_column (str): 
            The value column of the master and taxi_trips DataFrame.

    Returns:
        pd.DataFrame:
            The updated master data. If no new type appeared returns the original one.
    """
    max_id = master[id_column].max()
    list_new_values = [value for value in taxi_trips[value_column].values
                                  if value not in master[value_column].values]
    new_values_df = pd.DataFrame({
        value_column: range(max_id + 1, max_id + len(list_new_values) + 1),
        id_column: list_new_values
        })
    master.columns
    updated_master = pd.concat([master, new_values_df], ignore_index=True)
    
    return updated_master

In [55]:
test_payment_type_master = update_master(taxi_trips=taxi_trips_payment_type_only,
                                         master=payment_type_master,
                                         id_column='payment_type_id',
                                         value_column='payment_type'
                                         )

In [56]:
test_payment_type_master

,payment_type_id,payment_type
0,1,Prcard
1,2,Cash
2,3,Credit Card
3,4,Mobile
4,5,Unknown
5,6,No Charge
6,7,Dispute
7,X,8
8,Y,9


In [57]:
test_company_type_master = update_master(taxi_trips=taxi_trips_company_only,
                                         master=company_master,
                                         id_column='company_id',
                                         value_column='company'
                                         )

In [58]:
test_company_type_master

,company_id,company
0,1,City Service
1,2,Chicago Taxicab
2,3,Chicago Independents
3,4,Flash Cab
4,5,Sun Taxi
5,6,Choice Taxi Association
6,7,Taxi Affiliation Services
7,8,Medallion Leasin
8,9,5 Star Taxi
9,10,Star North Taxi Management Llc


#### update texi_trips with the most recent company_master and payment_type master codes

In [59]:
def update_taxi_trips_with_master_data(taxi_trips: pd.DataFrame,
                                   payment_type_master: pd.DataFrame,
                                   company_master: pd.DataFrame) -> pd.DataFrame:
    """ Removes the payment_type and the company columns, and adds their ids

    Args:
        taxi_trips (pd.DataFrame):
            DataFrame holding the daily taxi trips.
        payment_type_master (pd.Dataframe): Payment type master table.
        company_master (pd.Dataframe): Company master table.

    Returns:
        pd.DataFrame: The updated DataFrame having ids, not payment type and company names.
    """
    taxi_trips_id = taxi_trips.merge(payment_type_master, on='payment_type')
    taxi_trips_id = taxi_trips_id.merge(company_master, on='company')
    taxi_trips_id.drop(['payment_type', 'company'], axis=1, inplace=True)
    
    return taxi_trips_id

In [60]:
taxi_trips_id = update_taxi_trips_with_master_data(taxi_trips= taxi_trips,
                                   payment_type_master= payment_type_master,
                                   company_master= company_master)


In [61]:
taxi_trips_id.sample(5)

,trip_id,taxi_id,trip_start_timestamp,trip_end_timestamp,trip_seconds,trip_miles,pickup_community_area_id,dropoff_community_area_id,fare,tips,tolls,extras,trip_total,pickup_centroid_latitude,pickup_centroid_longitude,dropoff_centroid_latitude,dropoff_centroid_longitude,datetime_for_weather,payment_type_id,company_id
60,4296c4247ddc2ccb530233b8034ebce762b8c3bc,4ae32e2eb244ce143800e0c40055e537cc50e3358a07ce...,2023-11-30 16:00:00,2023-11-30T17:45:00.000,3009,7.08,38,8,27.5,0,0,0,27.5,41.812948939,-87.617859676,41.899602111,-87.633308037,2023-11-30 16:00:00,1,1
9714,a16154c656057305f7f6385373852f1a578042fc,5185bcbcb12036942b65e858a85af89bac955ee570ce66...,2023-11-30 11:00:00,2023-11-30T11:00:00.000,355,1.05,32,32,6.25,2,0,0,8.75,41.880994471,-87.632746489,41.870607372,-87.622172937,2023-11-30 11:00:00,3,14
8401,6e800d00b67df76972ba0973dde7d3c0db86e50f,8ef1056519939d511d24008e394f83e925d2539d668a00...,2023-11-30 14:00:00,2023-11-30T14:15:00.000,335,0,8,8,9,2.78,0,0,11.78,41.890922026,-87.618868355,41.890922026,-87.618868355,2023-11-30 14:00:00,4,9
7589,b4069ec6ab6050ef723d3438f77998faa9374b4b,fb16702bd1ce21e338b3135f8a3b83d434fda1b6b60cbf...,2023-11-30 08:00:00,2023-11-30T09:00:00.000,2242,15,5,56,38.42,8.05,0,0,46.47,41.947791586,-87.683834942,41.79259236,-87.769615453,2023-11-30 08:00:00,4,5
8565,d673a69aa3b5d37636c08ba3f845d971598d0589,1884386375a1f401bce15b26abbc119b734e1548e47fa3...,2023-11-30 19:00:00,2023-11-30T19:45:00.000,197,0.39,65,65,4.75,0,0,0,4.75,41.769778059,-87.726929842,41.769778059,-87.726929842,2023-11-30 19:00:00,2,14


#### weather transformation function

In [67]:
def transform_weather_data(weather_data: json) -> pd.DataFrame:
    """ Make transformations on the daily weather api response.

    Args:
        weather_data (json):
            The daily weather data from the Open Metao API.
        
    Returns:
        pd.DataFrame:
            A DataFrame representation of the weather data.
    """
    weather_data_filtered = {
        'datetime': weather_data['hourly']['time'],
        'temperature': weather_data['hourly']['temperature_2m'],
        'wind_speed': weather_data['hourly']['wind_speed_10m'],
        'rain': weather_data['hourly']['rain'],
        'precipitation': weather_data['hourly']['precipitation']
        }
    weather_df = pd.DataFrame(weather_data_filtered)
    weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])
    
    return weather_df

In [63]:
# test
url = 'https://archive-api.open-meteo.com/v1/era5'
# current date
current_datetime = datetime.now()
# get the data two months before as a formatted string
formatted_datetime = (current_datetime - relativedelta(months=2)).strftime('%Y-%m-%d')
params = {
    'latitude': 41.85,
    'longitude': -87.65,
    'start_date': formatted_datetime,
    'end_date': formatted_datetime,
    'hourly': 'temperature_2m,wind_speed_10m,rain,precipitation'
    }

response = requests.get(url, params=params)
weather_data = response.json()



In [68]:
weather_data_df = transform_weather_data(weather_data)

In [69]:
weather_data_df

,datetime,temperature,wind_speed,rain,precipitation
0,2023-11-29 00:00:00,-6.5,17.3,0.0,0.0
1,2023-11-29 01:00:00,-7.5,20.0,0.0,0.0
2,2023-11-29 02:00:00,-7.9,20.9,0.0,0.0
3,2023-11-29 03:00:00,-7.8,21.4,0.0,0.0
4,2023-11-29 04:00:00,-7.3,22.2,0.0,0.0
5,2023-11-29 05:00:00,-7.0,21.7,0.0,0.0
6,2023-11-29 06:00:00,-6.7,20.2,0.0,0.0
7,2023-11-29 07:00:00,-6.2,20.5,0.0,0.0
8,2023-11-29 08:00:00,-6.0,21.9,0.0,0.0
9,2023-11-29 09:00:00,-5.6,21.7,0.0,0.0
